In [ ]:
# Libraries we might need
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib.image as img
import PIL.Image as Image
import cv2

import os
import numpy as np
import pathlib

# I couldn't find the dataset link so please add it yourself
data_dir=''
data_dir=pathlib.Path(dir)
data_dir
## we are just taking 600 example of each so that training dont take alot of time
arborio = list(data_dir.glob('Arborio/*'))[:600]
basmati = list(data_dir.glob('Basmati/*'))[:600]
ipsala = list(data_dir.glob('Ipsala/*'))[:600]
jasmine = list(data_dir.glob('Jasmine/*'))[:600]
karacadag = list(data_dir.glob('Karacadag/*'))[:600]

## Lets print one image of each type of rice
arborio_image = img.imread(arborio[0])
basmati_image = img.imread(basmati[0])
ipsala_image = img.imread(ipsala[0])
jasmine_image = img.imread(jasmine[0])
karacadag_image = img.imread(karacadag[0])

fig, ax = plt.subplots(ncols=5, figsize=(20,5))
fig.suptitle('Rice Category')

ax[0].imshow(arborio_image)
ax[1].imshow(basmati_image)
ax[2].imshow(ipsala_image)
ax[3].imshow(jasmine_image)
ax[4].imshow(karacadag_image)

##lets create a dictionary to easy access the location and also gie each type a number from 0-4
df_images = {'arborio' : arborio,'basmati' : basmati,'ipsala' : ipsala,'jasmine' : jasmine,'karacadag': karacadag}
df_labels = {'arborio' : 0,'basmati' : 1,'ipsala' : 2,'jasmine' : 3,'karacadag': 4}

### lets check the dimension of a image (assuming other images are same as it is)
img = cv2.imread(str(df_images['arborio'][0])) 
img.shape

## lets convert the images to array and add to our input X and output y, we are resizing image to 224x224 because MobileNet is trained on 224x224
X, y = [], [] # X = images, y = labels
for label, images in df_images.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, (224, 224)) 
        X.append(resized_img) 
        y.append(df_labels[label])
## preprocessing the image
X = np.array(X)
X = X/255
y = np.array(y)
# splitting the data
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y ,test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.2, random_state=42)


# Creating the model
mobile_net = hub.KerasLayer('https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4', input_shape=(224,224, 3), trainable=False) 

num_label = 5

model = keras.Sequential([
    mobile_net,
    keras.layers.Dense(num_label,activation='softmax')])

## compile and train the model
model.compile(optimizer="adam",loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['acc'])

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Evaluate the model
model.evaluate(X_test,y_test)
## creating a classification_report
from sklearn.metrics import classification_report

y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))
